In [76]:
import testing
import training
import preprocessing
import importlib
from torchvision import transforms

import torch

importlib.reload(testing)
importlib.reload(training)
importlib.reload(preprocessing)

print('Libraries are loaded successfully')

Libraries are loaded successfully


In [9]:
# divide into two parts, train and valid.
preprocessing.build_file_structure('./dogs-vs-cats-redux-kernels-edition/train', 0.1)

# resize and nomalize image dataset
image_transform = transforms.Compose([
                             transforms.Resize((224, 224)),
                             transforms.ToTensor(),
                             transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                  std=[0.229, 0.224, 0.225])
                         ])

# make batch sets to boost training network
train_data_batches = preprocessing.dataset_load('./dogs-vs-cats-redux-kernels-edition/train/train', image_transform, 32, True, 3)
valid_data_batches = preprocessing.dataset_load('./dogs-vs-cats-redux-kernels-edition/train/valid', image_transform, 32, True, 3)

In [73]:
# Get resnet50 with 2 labels as output
model_ft = training.get_model()

# For convenience sake
datasets = {'train' : train_data_batches, 'valid' : valid_data_batches}

# Return the best model performed in valid dataset
final_model = training.training(model_ft, datasets, epochs=10)

----------------- Epoch 0 -----------------
train Error : 0.00723
train Correct : 0.90151
valid Error : 0.00491
valid Correct : 0.94680
Time : 2M 45s
----------------- Epoch 1 -----------------
train Error : 0.00467
train Correct : 0.93911
valid Error : 0.00520
valid Correct : 0.93280
Time : 2M 44s
----------------- Epoch 2 -----------------
train Error : 0.00431
train Correct : 0.94231
valid Error : 0.00746
valid Correct : 0.90760
Time : 2M 41s
----------------- Epoch 3 -----------------
train Error : 0.00353
train Correct : 0.95493
valid Error : 0.00527
valid Correct : 0.93520
Time : 2M 43s
----------------- Epoch 4 -----------------
train Error : 0.00303
train Correct : 0.96156
valid Error : 0.01034
valid Correct : 0.89480
Time : 2M 42s
----------------- Epoch 5 -----------------
train Error : 0.00286
train Correct : 0.96409
valid Error : 0.00499
valid Correct : 0.94760
Time : 2M 43s
----------------- Epoch 6 -----------------
train Error : 0.00253
train Correct : 0.96800
valid Erro

In [74]:
# Load best model
checkpoint = torch.load('dogs_vs_cats_classifier_res50.pth')
final_model = training.get_model()
final_model.load_state_dict(checkpoint)
final_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [77]:
# Build test dataset
test_data = testing.test_dataset('./dogs-vs-cats-redux-kernels-edition/test/*.jpg', image_transform)

# Predict test dataset by the best model
result = testing.testing(final_model, test_data)

# Export result as a csv file for submission
testing.export_submission(result)